In [1]:
%load_ext autoreload
%autoreload 2

In [106]:
import numbers

import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import hyclib as lib

In [4]:
arr = np.array([22, 41, 37])
shape = (7,6)
arr = np.unravel_index(arr, shape)
print(arr)
arr = np.ravel_multi_index(arr, shape)
print(arr)

(array([3, 6, 6]), array([4, 5, 1]))
[22 41 37]


In [5]:
t = torch.as_tensor(arr)
t = lib.pt.unravel_index(t, shape, as_tuple=True)
print(t)
print(lib.pt.ravel_multi_index(t, shape))

(tensor([3, 6, 6]), tensor([4, 5, 1]))
tensor([22, 41, 37])


In [175]:
torch.masked.MaskedTensor(torch.tensor([1,2,3]), torch.tensor([True, True, False])).max()

/Users/hoyinchau/opt/anaconda3/envs/hyclib/lib/python3.9/site-packages/torch/masked/maskedtensor/core.py:299: UserWarning: max is not implemented in __torch_dispatch__ for MaskedTensor.
If you would like this operator to be supported, please file an issue for a feature request at https://github.com/pytorch/maskedtensor/issues with a minimal reproducible code snippet.
In the case that the semantics for the operator are not trivial, it would be appreciated to also include a proposal for the semantics.
  warnings.warn(msg)


TypeError: no implementation found for 'torch._ops.aten.max.default' on types that implement __torch_dispatch__: [<class 'torch.masked.maskedtensor.core.MaskedTensor'>]

In [167]:
M, N = 100, 100000

# bins = [-2.5,-2,-1,0,1,2,3.0]
bins = 100
arr = np.random.normal(size=N)
indices = np.random.randint(N, size=M)
arr[indices] = np.nan

In [169]:
binnumbers, centers, edges = lib.sp.stats.bin(arr, bins=bins, nan_policy='raise')

t = torch.tensor(arr)
tbinnumbers, tcenters, tedges = lib.pt.stats.bin(t, bins=bins, nan_policy='raise')

pbinnumbers, pedges = pd.cut(arr, bins=bins, retbins=True, labels=False, right=False)

# print(binnumbers)
# print(tbinnumbers.numpy())
# print(pbinnumbers)
# print(centers)
# print(tcenters.numpy())
# print(binnumbers[np.isnan(arr)])
# print(tbinnumbers.numpy()[np.isnan(arr)])
# print(pbinnumbers[np.isnan(arr)])

torch.testing.assert_close(torch.from_numpy(binnumbers), tbinnumbers)
torch.testing.assert_close(torch.from_numpy(centers), tcenters, equal_nan=True)
torch.testing.assert_close(torch.from_numpy(edges), tedges)

isnan = np.isnan(pbinnumbers)
np.testing.assert_allclose(binnumbers[~isnan], pbinnumbers[~isnan] + 1)
if isinstance(bins, numbers.Number):
    r = np.nanmax(arr) - np.nanmin(arr)
    edges[-1] = edges[-1] + r * 0.001
np.testing.assert_allclose(edges, pedges)

ValueError: array([ 0.55059211, -0.55368352,  0.78836508, ...,  0.31585746,
        0.26697453,  0.64387578]) contains non-finite values.